In [1]:
import torch

In [2]:
dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")

Using cache found in /Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main
/Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [3]:
print(dinov2_vits14)

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [4]:
print(torch.hub.help("facebookresearch/dinov2", "dinov2_vits14"))


    DINOv2 ViT-S/14 model (optionally) pretrained on the LVD-142M dataset.
    


Using cache found in /Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main


In [5]:
from torchsummary import summary

In [6]:
summary(dinov2_vits14, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 384, 16, 16]         226,176
          Identity-2             [-1, 256, 384]               0
        PatchEmbed-3             [-1, 256, 384]               0
         LayerNorm-4             [-1, 257, 384]             768
            Linear-5            [-1, 257, 1152]         443,520
           Dropout-6          [-1, 6, 257, 257]               0
            Linear-7             [-1, 257, 384]         147,840
           Dropout-8             [-1, 257, 384]               0
   MemEffAttention-9             [-1, 257, 384]               0
       LayerScale-10             [-1, 257, 384]               0
        LayerNorm-11             [-1, 257, 384]             768
           Linear-12            [-1, 257, 1536]         591,360
             GELU-13            [-1, 257, 1536]               0
          Dropout-14            [-1, 25

In [7]:
import sys

sys.path.append("../")


from lib.datasets import ThingsMEGDatasetWithImages

import os

import hydra
import numpy as np
import torch
import torch.nn.functional as F
import wandb
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig
from termcolor import cprint
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from tqdm import tqdm

from lib.datasets import ThingsMEGDataset
from lib.models import BasicConvClassifier
from lib.utils import set_seed

data_dir = "../../data"
batch_size = 32
num_workers = 1
loader_args = {"batch_size": batch_size, "num_workers": num_workers}

train_set_with_images = ThingsMEGDatasetWithImages("train", data_dir)
train_loader_with_images = DataLoader(
    train_set_with_images, shuffle=False, **loader_args
)

In [8]:
for image_X, brain_X, y, subject_id in train_loader_with_images:
    print(image_X.shape, brain_X.shape, y.shape, subject_id)
    print(dinov2_vits14(image_X).shape)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32, 271, 281]) torch.Size([32]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([32, 384])


In [9]:
dinov2_vits14_lc = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14_lc")
print(dinov2_vits14_lc)

Using cache found in /Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_linear4_head.pth" to /Users/hayato/.cache/torch/hub/checkpoints/dinov2_vits14_linear4_head.pth
100%|██████████| 7.33M/7.33M [00:00<00:00, 8.38MB/s]

_LinearClassifierWrapper(
  (backbone): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=Fals

In [10]:
from torchsummary import summary

summary(dinov2_vits14_lc, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 384, 16, 16]         226,176
          Identity-2             [-1, 256, 384]               0
        PatchEmbed-3             [-1, 256, 384]               0
         LayerNorm-4             [-1, 257, 384]             768
            Linear-5            [-1, 257, 1152]         443,520
           Dropout-6          [-1, 6, 257, 257]               0
            Linear-7             [-1, 257, 384]         147,840
           Dropout-8             [-1, 257, 384]               0
   MemEffAttention-9             [-1, 257, 384]               0
       LayerScale-10             [-1, 257, 384]               0
        LayerNorm-11             [-1, 257, 384]             768
           Linear-12            [-1, 257, 1536]         591,360
             GELU-13            [-1, 257, 1536]               0
          Dropout-14            [-1, 25

In [2]:
import torch
model = torch.hub.load("pytorch/vision:v0.10.0", "resnet152", pretrained=True)
from torchsummary import summary

summary(model, (3, 224, 224))

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /Users/hayato/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /Users/hayato/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:04<00:00, 50.3MB/s] 


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,